In [1]:
import pandas as pd

In [2]:

import requests
from tqdm import tqdm
import re
import difflib

In [5]:
df1 = pd.read_pickle('G:/uiuc/timan/AIRC/data/parsed/2022-15372.pkl')
df2 = pd.read_pickle('G:/uiuc/timan/AIRC/data/parsed/2022-23918.pkl')


In [5]:
def get_header_to_text_mapping(df):
    ans_df = pd.DataFrame(columns = ['header', 'text'], dtype=object)
    txt = ""
    header = ""

    for i, row in tqdm(df.iterrows()):

        if row.tag == "HD":
            if i != 0:
                ans_df = ans_df.append({'header': re.sub(r'[^\w\s]','',header.lower()), 'text': re.sub(r'[^\w\s]','',txt.lower())}, ignore_index=True)
                txt = ""
            header = row.text

        elif row.tag == "P":
            txt = txt + " " + row.text
        # if i>1:break
    return ans_df

In [7]:
def get_string_changes(s1, s2):
    ans = []
    
    if type(s1) == str and type(s2) == str:
        n = 3
        s1 = [" ".join(s1.lower().split()[i:i+n]) for i in range(0, len(s1.split()), n)]
        s2 = [" ".join(s2.lower().split()[i:i+n]) for i in range(0, len(s2.split()), n)]

    diff = difflib.unified_diff(s1, s2, fromfile='Text 1', tofile='Text 2', lineterm='')
    for idx, line in enumerate(diff):
        if idx<3: continue
        ans.append(line)
            
    return ans

In [10]:
df1.head()

,xml_path,header,legal,tag,footnotes,numbering,text,cfr_title,cfr_part,section,section_subject,footnote
0,/NOTICE/PREAMB/AGY/HD,"(AGENCY:,)",False,HD,[],NaN,AGENCY:,NaN,NaN,NaN,NaN,NaN
1,/NOTICE/PREAMB/AGY/P,"(AGENCY:,)",False,P,[],NaN,"Maritime Administration, DOT.",NaN,NaN,NaN,NaN,NaN
2,/NOTICE/PREAMB/ACT/HD,"(ACTION:,)",False,HD,[],NaN,ACTION:,NaN,NaN,NaN,NaN,NaN
3,/NOTICE/PREAMB/ACT/P,"(ACTION:,)",False,P,[],NaN,Notice.,NaN,NaN,NaN,NaN,NaN
4,/NOTICE/PREAMB/SUM/HD,"(SUMMARY:,)",False,HD,[],NaN,SUMMARY:,NaN,NaN,NaN,NaN,NaN


In [8]:
# df1.text[df1.tag=="HD"]
doc1df=get_header_to_text_mapping(df1)

34it [00:00, 2000.51it/s]


In [9]:
doc2df=get_header_to_text_mapping(df2)


1140it [00:00, 5575.74it/s]


In [25]:
# a = (doc1df.text[doc1df.header=='summary'])
# b = (doc2df.text[doc2df.header=='summary'])

In [41]:
a='OPPS Update: For 2022, we are increasing the payment rates under the OPPS by an Outpatient Department (OPD) fee schedule increase factor of 2.0 percent. This increase factor is based on the proposed hospital inpatient market basket percentage increase of 2.7 percent for inpatient services paid under the hospital inpatient prospective payment system (IPPS) reduced by a proposed productivity adjustment of 0.7 percentage point. Based on this update, we estimate that total payments to OPPS providers (including beneficiary cost-sharing and estimated changes in enrollment, utilization, and case-mix) for calendar year (CY) 2022 would be approximately $82.078 billion, an increase of approximately $5.913 billion compared to estimated CY 2022 OPPS payments.'

In [42]:
b='OPPS Update: For 2023, we are increasing the payment rates under the OPPS by an Outpatient Department (OPD) fee schedule increase factor of 3.8 percent. This increase factor is based on the final hospital inpatient market basket percentage increase of 4.1 percent for inpatient services paid under the hospital inpatient prospective payment system (IPPS) reduced by a final productivity adjustment of 0.3 percentage point. Based on this update, we estimate that total payments to OPPS providers (including beneficiary cost-sharing and estimated changes in enrollment, utilization, and case-mix) for calendar year (CY) 2023 would be approximately $86.5 billion, an increase of approximately $6.5 billion compared to estimated CY 2022 OPPS payments.'

In [30]:
a=doc1df.iloc[2]['text']
b=doc2df.iloc[2]['text']

In [43]:
url = "https://text-diff.p.rapidapi.com/diff"

payload = {
	"text1": a,
	"text2": b
}
headers = {
	"content-type": "application/json",
	"X-RapidAPI-Key": "25711ed42fmsh972b785ee849dc3p1f1799jsnfb9f2fb98f69",
	"X-RapidAPI-Host": "text-diff.p.rapidapi.com"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"diffs":[{"type":"eq","text":"OPPS Update: For 202"},{"type":"del","text":"2"},{"type":"add","text":"3"},{"type":"eq","text":", we are increasing the payment rates under the OPPS by an Outpatient Department (OPD) fee schedule increase factor of "},{"type":"del","text":"2.0"},{"type":"add","text":"3.8"},{"type":"eq","text":" percent. This increase factor is based on the "},{"type":"del","text":"proposed"},{"type":"add","text":"final"},{"type":"eq","text":" hospital inpatient market basket percentage increase of "},{"type":"del","text":"2.7"},{"type":"add","text":"4.1"},{"type":"eq","text":" percent for inpatient services paid under the hospital inpatient prospective payment system (IPPS) reduced by a "},{"type":"del","text":"proposed"},{"type":"add","text":"final"},{"type":"eq","text":" productivity adjustment of 0."},{"type":"del","text":"7"},{"type":"add","text":"3"},{"type":"eq","text":" percentage point. Based on this update, we estimate that total payments to OPPS providers (inclu

In [44]:
import json
y = json.loads(response.text)

In [45]:
for i in y["diffs"]:
    if i["type"]=="eq":
        print(i["text"], end="")
    elif(i["type"]=="del"):
        print(colored(i["text"],'red'), end=" ")
    elif(i["type"]=="add"):
        print(colored(i["text"],'green'), end=" ")

OPPS Update: For 2022 3 , we are increasing the payment rates under the OPPS by an Outpatient Department (OPD) fee schedule increase factor of 2.0 3.8  percent. This increase factor is based on the proposed final  hospital inpatient market basket percentage increase of 2.7 4.1  percent for inpatient services paid under the hospital inpatient prospective payment system (IPPS) reduced by a proposed final  productivity adjustment of 0.7 3  percentage point. Based on this update, we estimate that total payments to OPPS providers (including beneficiary cost-sharing and estimated changes in enrollment, utilization, and case-mix) for calendar year (CY) 2022 3  would be approximately $82.078 6.5  billion, an increase of approximately $5.913 6.5  billion compared to estimated CY 2022 OPPS payments.

In [66]:
set1 = set(df1[df1.tag == "HD"].text)
set2 = set(df2[df2.tag == "HD"].text)
common_headers = set1.intersection(set2)


In [74]:
df1.loc[0,:]

xml_path           /NOTICE/PREAMB/AGY/HD
header                        (AGENCY:,)
legal                              False
tag                                   HD
footnotes                             []
numbering                            NaN
text                             AGENCY:
cfr_title                            NaN
cfr_part                             NaN
section                              NaN
section_subject                      NaN
footnote                             NaN
Name: 0, dtype: object

In [70]:
df1_headers = list(df1[df1.tag == "HD"].text)
df2_headers = list(df2[df2.tag == "HD"].text)

In [51]:
h1_idx, h2_idx = 0, 0

while True:
    if(df1.loc[id1, :].tag != "HD" and row2.tag == "HD"):
        print(row)

    if(row1.tag == "HD"):



        
    print(row)

xml_path           /NOTICE/PREAMB/AGY/HD
header                        (AGENCY:,)
legal                              False
tag                                   HD
footnotes                             []
numbering                            NaN
text                             AGENCY:
cfr_title                            NaN
cfr_part                             NaN
section                              NaN
section_subject                      NaN
footnote                             NaN
Name: 0, dtype: object
xml_path                    /NOTICE/PREAMB/AGY/P
header                                (AGENCY:,)
legal                                      False
tag                                            P
footnotes                                     []
numbering                                    NaN
text               Maritime Administration, DOT.
cfr_title                                    NaN
cfr_part                                     NaN
section                                     